In [1]:
import pandas as pd
import numpy as np

In [2]:
# Unificar tweets
tweets_k = pd.read_pickle('../datasets/TweetsDataset_K.pkl')[['screen_name','text','created_at']]
tweets_k['grupo'] = 'K'
tweets_m = pd.read_pickle('../datasets/TweetsDataset_M.pkl')[['screen_name','text','created_at']]
tweets_m['grupo'] = 'M'
tweets = pd.concat([tweets_k,tweets_m])

In [3]:
# Resumen del DataSet
print('K-Tweets_count:',tweets_k.screen_name.count())
print('K-Users_count:',tweets_k.screen_name.nunique())
print('M-Tweets_count:',tweets_m.screen_name.count())
print('M-Users_count:',tweets_m.screen_name.nunique())

K-Tweets_count: 252388
K-Users_count: 386
M-Tweets_count: 262108
M-Users_count: 411


### Sentiment Analysis

In [4]:
# Importar e Instanciar Sentiment Classifier
# Gracias y créðitos a https://github.com/aylliote/senti-py
from classifier import *
clf = SentimentClassifier()

/home/databuntu/afzjup/afzjup/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [5]:
# Función para iterar por la lista, predecir y anexar predicción como columna nueva y entregar dataframe resultante
def calc_sent(df):
    lista = []
    for index,row in df.iterrows():
        lista.append([row.grupo,row.created_at,row.screen_name,clf.predict(row.text),row.text])
    return pd.DataFrame(lista,columns=['grupo','created_at','screen_name','sent_score','text'])

In [6]:
from multiprocessing import Pool

In [7]:
def parallelize_dataframe(df, func, n_cores=4):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [8]:
tweets = parallelize_dataframe(tweets.head(15000),calc_sent,n_cores=28)

In [9]:
import openpyxl
tweets.to_excel('../datasets/tweets_M_K.xlsx',index=False)